# AMGeO API Release

<img src='./static/AMGeOLogo.svg'/>

Welcome to the AMGeO API release notebook, going over the API and how to use it.

### TODO: different ways to run this notebook

### Special thanks to all of our data providers!

- [SuperMAG](https://supermag.jhuapl.edu/)
- [SuperDARN](http://vt.superdarn.org/tiki-index.php)
- [AMPERE](http://ampere.jhuapl.edu/)
- [NASA SPDF](https://spdf.gsfc.nasa.gov/)

# AMGeO's new API

## Importing the API

To import AMGeO's Python API, execute the below. 

<mark>NOTE:</mark> upon import, if you have not configured AMGeO with your API key, SuperMAG username or AMPERE username, this will be asked here (with instructions on how to get each).

In [1]:
from AMGeO.api import AMGeOApi

Solar wind data files will be saved to /Users/willemmirkovich/Library/Application Support/nasaomnireader

------------IMPORTANT----------------------------
Unable to import spacepy. Will fall back to
using Omni text files, which may have slightly
different data and incomplete metadata
-------------------------------------------------



## Creating an AMGeO API instance

Here, we can create an AMGeO API instance, and get various details as to a default API instance

In [2]:
api = AMGeOApi()
api

AMGeO API instance
current configuration: default
current output directory: /Users/willemmirkovich/AMGeO/AMGeO-API-Release/amgeo_out

### Setting our output directory for AMGeO data

When running AMGeO, a local file system directory will be selected to store your assimilative maps. By default, AMGeo will use ```~/amgeo_v2_ouptput```

In [3]:
api.get_output_dir()

'/Users/willemmirkovich/AMGeO/AMGeO-API-Release/amgeo_out'

But, if you want to specify another directory, you can set this in the API instance using ```set_output_dir```

In [4]:
api.set_output_dir('./amgeo_out')

In [5]:
api.get_output_dir()

'/Users/willemmirkovich/AMGeO/AMGeO-API-Release/amgeo_out'

# Creating assimilative maps using AMGeO's new API

Now that we have an AMGeO API instance, we can start creating new assimilative map data!

## Creating an AMGeO Controller

To generate assimilative maps, you will have to load specific settings from AMGeO. Conveniently, AMGeO's new API allows for a simple way to load AMGeO's default settings using a ```controller```.

To create a ```controller``` instance, you can call ```get_controller``` on an API instance

In [6]:
controller = api.get_controller()
controller

Default AMGeO Controller
Creates Assimilative Maps using SuperMAG and SuperDARN observations
Does not predict magnetic potential nor field-aligned current

As observerd, calling ```get_controller``` returns an ```Default AMGeO Controller```, that is able to both create and load assimilative maps using AMGeO's default settings. 

## Generating AMGeO maps

<img src="./static/AMGeOElectricPotentialMap.png" width="500px" height="500px">

Now that we have a ```controller``` instance, we can create assimilative maps by calling the ```generate``` method on our ```controller```. This method takes both a hemisphere and a date argument. For example, if we wanted to generate maps for dates:

- January 6th, 2013 12:30:00
- January 6th, 2013 13:30:00
- February 6th, 2013 12:30:00
- February 6th, 2013 13:30:00

and on the Northern hemisphere, we can call ```generate``` with ...

In [7]:
# datetime module is used for passing dates/datetimes
from datetime import datetime, date

In [8]:
dts = [
    datetime(2013, 1, 6, 12, 30, 0), # January 6th, 2013 12:30:00
    datetime(2013, 1, 6, 13, 30, 0), # January 6th, 2013 13:30:00
    datetime(2013, 2, 6, 12, 30, 0), # February 6th, 2013 12:30:00
    datetime(2013, 2, 6, 13, 30, 0) # February 6th, 2013 13:30:00
]
# hemisphere
h = 'N'

In [9]:
controller.generate(dts, h)

Data requested for 2 day(s)


Request recieved for 2013-1-6 N
No new datetimes to complete for 2013-01-06 12:30:00
2013-1-6 N complete
Request recieved for 2013-2-6 N
No new datetimes to complete for 2013-02-06 12:30:00
2013-2-6 N complete

AMGeO complete


Once complete, we will be able to see generated AMGeO maps for each datetime.

```controller.generate``` supports multiple different ways of generating maps based on dates/datetimes:

1. A single datetime

    Will generate data for a specific date and time 
    ```python
    controller.generate(datetime(YYYY, MM, DD, hh, mm, ss), 'N' | 'S')
    ```
    
2. A single date

    Will generate data for 5 min slices over the entire date provided
    # TODO: need to fix these to correct time slices
    Ex: date(2013, 5, 5) => datetime(2013, 5, 5, 0, 2, 30), datetime(2013, 5, 5, 0, 2, 30)
    ```python
    controller.generate(date(YYYY, MMMM, DD), 'N' | 'S')
    ```
3. A list of dates/datetimes

    This will handle each element within the list on a case by case basis, in a bulk job
    ```python
    controller.generate([
        datetime(YYYY, MM, DD, hh, mm, ss),
        date(YYYY, MM, DD),
        ...
    ], 'N' | 'S')
    ```

## Browsing AMGeO maps

Once we have generated AMGeO maps, we might be interested in browsing what maps we have available.

To do this, the ```controller``` has a ```browse``` method that can be used in two ways.

1. Browse based on a hemisphere which dates have AMGeO maps already generated.

    ```python
    controller.browse('N' | 'S')
    ```

In [10]:
controller.browse('N')

[datetime.date(2013, 1, 6), datetime.date(2013, 2, 6)]

2. Specify a date and hemisphere for specific times that AMGeO has generated maps for

    ```python
    controller.browse(date(YYYY, MM, DD), 'N' | 'S')
    ```

In [11]:
controller.browse(date(2013, 2, 6), 'N')

[datetime.datetime(2013, 2, 6, 12, 30), datetime.datetime(2013, 2, 6, 13, 30)]

## Loading AMGeO maps

The last and most important piece of AMGeO's new API is the ability to load AMGeO maps into [Xarray datasets](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html), with no work needed other than calling ```controller.load```

```load``` supports the same modularity as ```generate```, to allow for you load various dates/datetimes on a given hemisphere into one dataset.

1. A single datetime

    Will load the specific date into a dataset
    ```python
    controller.load(datetime(YYYY, MM, DD, hh, mm, ss), 'N' | 'S')
    ```

In [12]:
controller.load(datetime(2013, 1, 6, 12, 30, 0), 'N')

<xarray.Dataset>
Dimensions:         (lat: 24, lon: 37, time: 1)
Coordinates:
  * time            (time) datetime64[ns] 2013-01-06T12:30:00
  * lat             (lat) float64 88.33 86.67 85.0 83.33 ... 53.33 51.67 50.0
  * lon             (lon) float64 0.0 10.0 20.0 30.0 ... 330.0 340.0 350.0 360.0
Data variables:
    E_ph            (time, lat, lon) float64 0.007483 0.009203 ... 7.415e-06
    E_th            (time, lat, lon) float64 0.02322 0.02402 ... -0.0003566
    cond_hall       (time, lat, lon) float64 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0 4.0
    cond_ped        (time, lat, lon) float64 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0 4.0
    epot            (time, lat, lon) float64 -7.467e+03 -7.787e+03 ... 406.6
    int_joule_heat  (time) float64 9.685
    joule_heat      (time, lat, lon) float64 2.38 2.647 ... 0.0004378 0.0005088
    v_ph            (time, lat, lon) float64 395.1 408.8 ... -0.6668 -7.301
    v_th            (time, lat, lon) float64 -127.3 -156.6 ... -6.742 -0.1518
Attributes:
    description:  AMGeO Assimilative Maps
    version:      v2_beta
    hemisphere:   N

2. A single date

    Will load all maps availabe from a date
    ```python
    controller.load(date(YYYY, MM, DD), 'N' | 'S')
    ```

In [13]:
controller.load(date(2013, 1, 6), 'N')

<xarray.Dataset>
Dimensions:         (lat: 24, lon: 37, time: 2)
Coordinates:
  * time            (time) datetime64[ns] 2013-01-06T12:30:00 2013-01-06T13:3...
  * lat             (lat) float64 88.33 86.67 85.0 83.33 ... 53.33 51.67 50.0
  * lon             (lon) float64 0.0 10.0 20.0 30.0 ... 330.0 340.0 350.0 360.0
Data variables:
    E_ph            (time, lat, lon) float64 0.007483 0.009203 ... 2.73e-06
    E_th            (time, lat, lon) float64 0.02322 0.02402 ... -0.0003534
    cond_hall       (time, lat, lon) float64 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0 4.0
    cond_ped        (time, lat, lon) float64 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0 4.0
    epot            (time, lat, lon) float64 -7.467e+03 -7.787e+03 ... 411.4
    int_joule_heat  (time) float64 9.685 7.434
    joule_heat      (time, lat, lon) float64 2.38 2.647 ... 0.0004222 0.0004996
    v_ph            (time, lat, lon) float64 395.1 408.8 ... -0.7162 -7.237
    v_th            (time, lat, lon) float64 -127.3 -156.6 ... -6.614 -0.0559
Attributes:
    description:  AMGeO Assimilative Maps
    version:      v2_beta
    hemisphere:   N

3. A list of dates/datetimes

    Will load each date/datetime respectively from the list. NOTE: you can load from multiple dates into one dataset
    ```python
    controller.load([
        datetime(YYYY, MM, DD, hh, mm, ss),
        date(YYYY, MM, DD),
        ...
    ], 'N' | 'S')
    ```

In [14]:
controller.load([
    date(2013, 1, 6),
    datetime(2013, 2, 6, 12, 30, 0)
], 'N')

<xarray.Dataset>
Dimensions:         (lat: 24, lon: 37, time: 3)
Coordinates:
  * time            (time) datetime64[ns] 2013-01-06T12:30:00 ... 2013-02-06T...
  * lat             (lat) float64 88.33 86.67 85.0 83.33 ... 53.33 51.67 50.0
  * lon             (lon) float64 0.0 10.0 20.0 30.0 ... 330.0 340.0 350.0 360.0
Data variables:
    E_ph            (time, lat, lon) float64 0.007483 0.009203 ... 2.164e-06
    E_th            (time, lat, lon) float64 0.02322 0.02402 ... -0.0003038
    cond_hall       (time, lat, lon) float64 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0 4.0
    cond_ped        (time, lat, lon) float64 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0 4.0
    epot            (time, lat, lon) float64 -7.467e+03 -7.787e+03 ... 343.2
    int_joule_heat  (time) float64 9.685 7.434 5.876
    joule_heat      (time, lat, lon) float64 2.38 2.647 ... 0.0003162 0.0003692
    v_ph            (time, lat, lon) float64 395.1 408.8 ... -0.6552 -6.221
    v_th            (time, lat, lon) float64 -127.3 -156.6 ... -5.72 -0.04431
Attributes:
    description:  AMGeO Assimilative Maps
    version:      v2_beta
    hemisphere:   N

As you may have noticed, it is really easy to plug dates from the ```browse``` method into the ```load``` method

In [15]:
hemi = 'N'
dates = controller.browse(hemi)
controller.load(dates, hemi)

<xarray.Dataset>
Dimensions:         (lat: 24, lon: 37, time: 4)
Coordinates:
  * time            (time) datetime64[ns] 2013-01-06T12:30:00 ... 2013-02-06T...
  * lat             (lat) float64 88.33 86.67 85.0 83.33 ... 53.33 51.67 50.0
  * lon             (lon) float64 0.0 10.0 20.0 30.0 ... 330.0 340.0 350.0 360.0
Data variables:
    E_ph            (time, lat, lon) float64 0.007483 0.009203 ... 3.771e-07
    E_th            (time, lat, lon) float64 0.02322 0.02402 ... -0.0003171
    cond_hall       (time, lat, lon) float64 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0 4.0
    cond_ped        (time, lat, lon) float64 4.0 4.0 4.0 4.0 ... 4.0 4.0 4.0 4.0
    epot            (time, lat, lon) float64 -7.467e+03 -7.787e+03 ... 363.5
    int_joule_heat  (time) float64 9.685 7.434 5.876 6.996
    joule_heat      (time, lat, lon) float64 2.38 2.647 ... 0.0003407 0.0004021
    v_ph            (time, lat, lon) float64 395.1 408.8 ... -0.6923 -6.492
    v_th            (time, lat, lon) float64 -127.3 -156.6 ... -5.936 -0.007721
Attributes:
    description:  AMGeO Assimilative Maps
    version:      v2_beta
    hemisphere:   N

## Xarray Datasets

This section is to go over some of the neat features that come with interacting with AMGeO maps with Xarray datasets

Below is the organization of an Xarray dataset with AMGeO maps

![AMGeO Xarray dataset](./static/AMGeOXarrayDataset.png)

### Data Variables

### Coordinates/Dimensions

### Metadata

### Plotting

### Numpy compatability

### Pandas compatability

# Conclusion

Thanks for checking out this notebook going over AMGeO's new API! 

Please get started with AMGeO [here](https://amgeo.colorado.edu/)

TODO: fill in